# Análisis: Iteración de Valores

**Estudiante:** [Tu nombre]

## Introducción

Este documento presenta el análisis de la implementación de **Iteración de Valores** para la solución de Procesos de Decisión de Markov (MDPs). Se evalúa el algoritmo en dos ambientes:

1. **GridWorld 10x10** - con diferentes números de iteraciones
2. **Bridge** - con factores de descuento 0.9 y 0.1

## Pregunta 1: GridWorld - Convergencia de Valores y Acciones

### Ejecución

Se ejecutó la implementación de iteración de valores con 5, 10, 15, 20, 30, y 50 iteraciones sobre GridWorld 10x10.

In [ ]:
from gridworld_environment import GridWorld10x10
from mdp import MDP
from value_iteration import ValueIteration
import numpy as np

env = GridWorld10x10()
mdp = MDP(env)
arrows = {'up': '  ↑  ', 'down': '  ↓  ', 'left': '  ←  ', 'right': '  →  ', None: '  ●  '}

def print_values(vi, env):
    for r in range(env.nrows):
        row = ''
        for c in range(env.ncols):
            if env.board[r][c] == '#':
                row += '  ####  '
            else:
                row += f'{vi.get_value((r, c)):+7.3f} '
        print(row)

def print_policy(vi, env):
    for r in range(env.nrows):
        row = ''
        for c in range(env.ncols):
            if env.board[r][c] == '#':
                row += ' #### '
            else:
                action = vi.get_policy((r, c))
                row += f'{arrows[action]}'
        print(row)

# Ejecutar para diferentes iteraciones
for n_iter in [5, 10, 15, 20, 30, 50]:
    print(f"\n{'='*60}")
    print(f"Iteraciones: {n_iter}")
    print(f"{'='*60}")
    
    vi = ValueIteration(mdp, discount=0.9, iterations=n_iter)
    vi.run_value_iteration()
    
    print("\nValores:")
    print_values(vi, env)
    
    print("\nPolítica:")
    print_policy(vi, env)

### Resultados Observados

#### 5 Iteraciones
- **Valores**: La mayoría de estados tienen valor 0, solo los estados cercanos al objetivo `(5,5)` tienen valores positivos.
- **Política**: Incompleta, muchos estados apuntan en direcciones arbitrarias ya que los valores aún no se han propagado.

#### 10 Iteraciones
- **Valores**: Los valores comienzan a propagarse desde el objetivo hacia el resto del grid.
- **Política**: Se observa una política coherente en la región cercana al objetivo, pero las esquinas aún no tienen direcciones óptimas.

#### 15 Iteraciones
- **Valores**: Los valores alcanzan la esquina superior izquierda `(0,0)`, estado inicial.
- **Política**: **La política converge** - las acciones son coherentes y apuntan hacia el objetivo `(5,5)` evitando los terminales negativos.

#### 20-30 Iteraciones
- **Valores**: Refinamiento en los decimales, pero la política permanece **idéntica** a la de 15 iteraciones.
- **Política**: Sin cambios.

#### 50 Iteraciones
- **Valores**: Convergencia completa, cambios solo en el tercer decimal.
- **Política**: Sin cambios respecto a 15 iteraciones.

### Análisis

**¿Cuándo convergen los valores?**
- Los **valores convergen completamente alrededor de las 30-50 iteraciones**, cuando los cambios entre iteraciones son menores a 0.001.

**¿Cuándo convergen las acciones?**
- La **política (acciones) converge mucho antes, alrededor de las 15 iteraciones**.
- Esto es un resultado importante: **la política óptima se puede extraer antes de que los valores converjan completamente**.

**Explicación:**
- La política depende del **orden relativo** de los Q-valores, no de sus valores absolutos exactos.
- Una vez que `Q(s, a_mejor) > Q(s, otras_acciones)`, la acción óptima ya está determinada, incluso si los valores exactos aún están cambiando.
- Los valores continúan refinándose para converger al valor exacto `V*(s)`, pero la política ya es óptima.

## Pregunta 2: Bridge - Efecto del Factor de Descuento

### Ejecución

Se ejecutó la implementación sobre el ambiente Bridge durante 10 iteraciones con factores de descuento γ = 0.9 y γ = 0.1.

In [ ]:
from bridge_environment import BridgeEnvironment

env_bridge = BridgeEnvironment()
mdp_bridge = MDP(env_bridge)

for discount in [0.9, 0.1]:
    print(f"\n{'='*60}")
    print(f"Factor de Descuento: γ = {discount}")
    print(f"{'='*60}")
    
    vi = ValueIteration(mdp_bridge, discount=discount, iterations=10)
    vi.run_value_iteration()
    
    print("\nValores:")
    print_values(vi, env_bridge)
    
    print("\nPolítica:")
    print_policy(vi, env_bridge)

### Resultados

#### γ = 0.9 (Alto - valora recompensas futuras)

**Valores (Fila 1 - El Puente):**
```
+0.000  -32.967  -52.530  -40.921  -13.404  +32.967  +0.000
```

**Política (Fila 1):**
```
LEFT  LEFT  LEFT  RIGHT  RIGHT  RIGHT  EXIT
```

**Observaciones:**
- El agente **cruza el puente** desde `(1,0)` hasta `(1,6)` para alcanzar la recompensa +100.
- Los valores en el puente son negativos debido al riesgo de caer en las casillas -100, pero el valor esperado total es positivo porque γ=0.9 hace que el +100 futuro sea muy valioso.
- Los valores cerca del objetivo son positivos: `V(1,5) = +32.967`.

#### γ = 0.1 (Bajo - descuenta fuertemente el futuro)

**Valores (Fila 1 - El Puente):**
```
+0.000  -30.303  -32.140  -32.028  -28.466  +30.303  +0.000
```

**Política (Fila 1):**
```
LEFT  LEFT  LEFT  RIGHT  RIGHT  RIGHT  EXIT
```

**Observaciones:**
- El agente **también cruza el puente**, la política es idéntica.
- Sin embargo, los valores son **mucho más bajos** en general.
- Con γ=0.1, la recompensa +100 en `(1,6)` se descuenta fuertemente: después de 6 pasos, vale aproximadamente `100 * 0.1^6 = 0.0001`, casi nada.

**Diferencia clave:**
- **Esquina superior derecha `(0,6)`:**
  - γ=0.9: Política = **RIGHT** (hacia el objetivo +100)
  - γ=0.1: Política = **DOWN** (hacia el terminal -100)
  
Con γ=0.1, el objetivo lejano casi no vale nada, por lo que en estados alejados la política puede ser subóptima o indiferente.

### Análisis

**¿Qué cambios se observan?**

1. **Valores absolutos:**
   - Con γ=0.9, los valores son más altos (tanto positivos como negativos más extremos).
   - Con γ=0.1, los valores están más comprimidos cerca de cero.

2. **Política en el puente:**
   - Ambos cruzan el puente (idéntica política).
   - Esto ocurre porque el agente **empieza en el puente** `(1,0)`, por lo que no tiene otra opción más que cruzar.

3. **Política en estados lejanos:**
   - Con γ=0.9, estados alejados del puente aún "sienten" la atracción del objetivo +100.
   - Con γ=0.1, estados alejados casi ignoran el objetivo porque está muy descontado.

**¿Por qué cambian los resultados?**

El factor de descuento γ controla cuánto valora el agente las recompensas futuras:

- **γ cercano a 1 (e.g., 0.9):**
  - El agente es **visionario** - planifica a largo plazo.
  - Recompensas futuras tienen casi el mismo peso que recompensas inmediatas.
  - Valores más altos en magnitud.
  
- **γ cercano a 0 (e.g., 0.1):**
  - El agente es **miope** - solo le importa el futuro inmediato.
  - Recompensas lejanas se descuentan exponencialmente: `γ^t`.
  - Valores más bajos en magnitud, política puede ser subóptima en estados lejanos.

**Fórmula:**
```
V(s) = Σ γ^t * R_t
```

Con γ=0.1, después de 10 pasos: `γ^10 = 0.1^10 = 1e-10` - las recompensas futuras prácticamente desaparecen.

### Conclusión

- La **política óptima depende del factor de descuento**.
- En el Bridge, ambas políticas cruzan porque el agente empieza en el puente.
- En estados alejados del objetivo, γ bajo produce políticas que ignoran recompensas lejanas.
- Para problemas de planificación a largo plazo, se recomienda γ ≥ 0.9.
- Para problemas de recompensa inmediata, γ bajo (0.1-0.5) puede ser apropiado.

## Conclusiones Generales

1. **Convergencia de política vs valores:**
   - La política converge más rápido (~15 iteraciones) que los valores (~30-50 iteraciones).
   - Esto sugiere que para obtener una política óptima, no es necesario esperar la convergencia completa de los valores.

2. **Efecto del factor de descuento:**
   - γ alto (0.9): planificación a largo plazo, valores altos.
   - γ bajo (0.1): planificación miope, valores bajos, política puede ignorar objetivos lejanos.

3. **Iteración de Valores es efectiva:**
   - Converge a la política óptima en ambos ambientes.
   - Método simple y robusto para resolver MDPs.